In [1]:
import pandas as pd
import requests
import glob
from datetime import datetime

## Extracting Data using API

In [4]:
url = "https://api.apilayer.com/exchangerates_data/latest?base=EUR&apikey=qvY7W8m90WrZ7eTzioXmpQRxMBZgB4rA" 
r=requests.get(url)
df=pd.DataFrame(r.json())['rates']
df.columns=["Currency","Rate"]
df.to_csv('exchange_rate_api.csv')

### Download Data using wget function

In [5]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json

'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process,index_col=0)
    return dataframe


### Extract function


In [7]:
def extract():
    extracted_data=pd.DataFrame(columns=['Name','cap_USD'])
    
    for csvfile in glob.glob("bank1.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile),ignore_index=True)
    
    return extracted_data

In [8]:
ex_rate=pd.read_csv('exchange_rate_api.csv',index_col=0)    #download csv file in same directory
ex_rate_to_gbp=ex_rate['rates']['GBP']

### Transform function


In [9]:
def transform(data):
    data.cap_USD=data.cap_USD.astype(float)
    data['cap_USD']=round(data.cap_USD*ex_rate_to_gbp,2)
    data.rename(columns={'cap_USD':'cap_GBP'},inplace=True)
    return data 
    
    

### load function

In [10]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  

### log function

In [11]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## Running ETL Process

In [12]:
log("ETL Job Started")

In [13]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

C:\Users\Gaurav Verma\AppData\Local\Temp\ipykernel_11700\3882947496.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile),ignore_index=True)


,Name,cap_USD
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399
...,...,...
100,Ping An Bank,37.993
101,Standard Chartered,37.319
102,United Overseas Bank,35.128
103,QNB Group,33.56


In [14]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data

,Name,cap_GBP
0,JPMorgan Chase,333.68
1,Industrial and Commercial Bank of China,294.65
2,Bank of America,277.68
3,Wells Fargo,262.90
4,China Construction Bank,219.70
...,...,...
100,Ping An Bank,32.43
101,Standard Chartered,31.85
102,United Overseas Bank,29.98
103,QNB Group,28.64


In [15]:
log("Load phase Started")
load('Exchange_rate_GBP',transformed_data)
log("Load phase Ended")

In [16]:
log("ETL Job Ended")